<a href="https://colab.research.google.com/github/MariamZayed/Bioinformatics_Labs/blob/main/Lab8_Bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**In This Lab**

Like the previous lab you will dawnload and imported two libs


you can find them here

- InitializeMatrix:

https://medmansedu-my.sharepoint.com/personal/eman_elsaeed_mans_edu_eg/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Feman_elsaeed_mans_edu_eg%2FDocuments%2FInitializeMatrix%2Epy&parent=%2Fpersonal%2Feman_elsaeed_mans_edu_eg%2FDocuments

- PrintMatrix

https://medmansedu-my.sharepoint.com/personal/eman_elsaeed_mans_edu_eg/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Feman_elsaeed_mans_edu_eg%2FDocuments%2FPrintMatrix%2Epy&parent=%2Fpersonal%2Feman_elsaeed_mans_edu_eg%2FDocuments

##**Importing Section**

In [28]:
pip install biopython

In [29]:
import Bio 
from Bio.Seq import Seq
from Bio import SeqIO #this lib to read and itra files
from Bio import motifs #for creating motifes objects
import InitializeMatrix #dawnload this library from the link above and then imported it like now
import PrintMatrix #dawnload this library from the link above and then imported it here

##**HMM**

###Initializing paramter for the upcoming algorithms

In [30]:
states = ['b','y','n','e']
print(states)

['b', 'y', 'n', 'e']


__________

defining transistion values

In [31]:
transitions = {('b','y') : 0.2,
               ('b','n') : 0.8,
               ('y','y') : 0.7,
               ('y','n') : 0.2,
               ('y','e') : 0.1,
               ('n','n') : 0.8,
               ('n','y') : 0.1,
               ('n','e') : 0.1 
               }
print(transitions)

{('b', 'y'): 0.2, ('b', 'n'): 0.8, ('y', 'y'): 0.7, ('y', 'n'): 0.2, ('y', 'e'): 0.1, ('n', 'n'): 0.8, ('n', 'y'): 0.1, ('n', 'e'): 0.1}


------------

defining emission values

In [32]:
emissions = {'y' : {'A':0.1, 'C':0.4, 'G':0.4, 'T':0.1},
             'n' : {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25} 
             }
print(emissions)

{'y': {'A': 0.1, 'C': 0.4, 'G': 0.4, 'T': 0.1}, 'n': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}}


-----------

preparing the sequence

In [33]:
sequence = 'ATGCG'
print(sequence)

ATGCG


##Backward Algorithm

###Intializing the Backward Matrix

In [34]:
F = InitializeMatrix.initialize_matrix(len(states), len(sequence) + 2)
F[-1][-1] = 1
PrintMatrix.print_matrix(F, states, sequence)

          0         A         T         G         C         G         0         
b         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
y         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
n         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
e         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  


> - note that we wanted the last index in the backward algorithm, so we are calling F[-1][-1]
> -the last row last column cell value is now equals 1 as we wanted

------------

###work with first column G

First column to start with has special equation than other columns

In [35]:
for i in range(1, len(states) - 1):
  F[i][-2] = transitions[(states[i], states[-1])]
PrintMatrix.print_matrix(F, states, sequence)

          0         A         T         G         C         G         0         
b         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
y         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e-01  0.00e+00  
n         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e-01  0.00e+00  
e         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  


-----------

###rest of the columns

Now we will compute from the second last to the first, we calculate the values as the sum of probabilities


In [36]:
for j in range(len(sequence)-1,0,-1): # loops on the symbols 
  for i in range(1,len(states)-1): # loops on the states 
    p_sum = 0
    for k in range(1,len(states)-1): # loops on all of the possible successive states 
      p_sum += F[k][j+1]*transitions[(states[i],states[k])]*emissions[states[k]][sequence[j]]
      F[i][j] = p_sum
PrintMatrix.print_matrix(F, states, sequence)

          0         A         T         G         C         G         0         
b         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
y         0.00e+00  3.08e-04  3.23e-03  1.04e-02  3.30e-02  1.00e-01  0.00e+00  
n         0.00e+00  3.61e-04  1.64e-03  6.12e-03  2.40e-02  1.00e-01  0.00e+00  
e         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  


________

###finally, the probability value 

Now, we need to calculate the final value. It is the sum of probabilities

you will find the final value in the first cell of the table


In [38]:
p_sum = 0
for k in range(1,len(states)-1):
  p_sum += F[k][1]*transitions[(states[0],states[k])]*emissions[states[k]][sequence[0]]
  F[0][0] = p_sum
PrintMatrix.print_matrix(F, states, sequence)

          0         A         T         G         C         G         0         
b         7.83e-05  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  
y         0.00e+00  3.08e-04  3.23e-03  1.04e-02  3.30e-02  1.00e-01  0.00e+00  
n         0.00e+00  3.61e-04  1.64e-03  6.12e-03  2.40e-02  1.00e-01  0.00e+00  
e         0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  2.00e-02  
